### Kaggle Configuration

In [21]:
import os
from pathlib import Path

is_kaggle = "KAGGLE_WORKING_DIR" in os.environ or "/kaggle" in os.getcwd()
print("Running on Kaggle:", is_kaggle)

if is_kaggle:
    path = Path("/kaggle/input/us-patent-phrase-to-phrase-matching")
    ! pip install -q datasets
else:
    path = Path(os.getcwd())

Running on Kaggle: False


In [22]:
import pandas as pd

df = pd.read_csv(path/"train.csv")

A good starting point within any Kaggle competition is to check the what our data consists of. To do this we should:

1. Print out the data frame
2. Read the [Dataset description](https://www.kaggle.com/competitions/us-patent-phrase-to-phrase-matching/data)
3. Call dataframe's `describe()` method

In [23]:
df

,id,anchor,target,context,score
0,37d61fd2272659b1,abatement,abatement of pollution,A47,0.50
1,7b9652b17b68b7a4,abatement,act of abating,A47,0.75
2,36d72442aefd8232,abatement,active catalyst,A47,0.25
3,5296b0c19e1ce60e,abatement,eliminating process,A47,0.50
4,54c1e3b9184cb5b6,abatement,forest region,A47,0.00
...,...,...,...,...,...
36468,8e1386cbefd7f245,wood article,wooden article,B44,1.00
36469,42d9e032d1cd3242,wood article,wooden box,B44,0.50
36470,208654ccb9e14fa3,wood article,wooden handle,B44,0.50
36471,756ec035e694722b,wood article,wooden material,B44,0.75


In [24]:
df.describe(include="object")

,id,anchor,target,context
count,36473,36473,36473,36473
unique,36473,733,29340,106
top,37d61fd2272659b1,component composite coating,composition,H01
freq,1,152,24,2186


We're going to do a lot with not much unique data. There's a lot of repetition and each entry only has 3-4 words.

Next we want to create an input column for our NLP model to read that combines categorizes and combines all our text columns in to one string we'll input in to the mode;.

In [25]:
df["input"] = "TEXT1: " + df.context + "; TEXT2: " + df.target + "; ANC1: " + df.anchor
df.input.head()

0    TEXT1: A47; TEXT2: abatement of pollution; ANC...
1    TEXT1: A47; TEXT2: act of abating; ANC1: abate...
2    TEXT1: A47; TEXT2: active catalyst; ANC1: abat...
3    TEXT1: A47; TEXT2: eliminating process; ANC1: ...
4    TEXT1: A47; TEXT2: forest region; ANC1: abatement
Name: input, dtype: object

## Tokenization

Machine Learning models operate on numbers not text. We need to convert our input in to numbers. To do this we need to do two things:
1. Tokenization: Split each text up in to tokens
2. Numericalization: Convert each token in to a number

Transformers use Datasets for storing data

In [26]:
from datasets import Dataset, DatasetDict
ds = Dataset.from_pandas(df)
ds

Dataset({
    features: ['id', 'anchor', 'target', 'context', 'score', 'input'],
    num_rows: 36473
})

How to tokenize & numericalize text varys between different models

In [27]:
model_name = "microsoft/deberta-v3-small"

Tokens aren't necessarily words as we need to be able to handle text that isn't made up of words such as URLs and we need to limit the size of our vocabularly so less common words will be split up.

In [41]:
from transformers import AutoModelForSequenceClassification,AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.tokenize("This is Declan's tokenizer")

/Users/Declan/micromamba/envs/beginner_nlp/lib/python3.10/site-packages/transformers/convert_slow_tokenizer.py:473: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


['▁This', '▁is', '▁Declan', "'", 's', '▁token', 'izer']

In [29]:
def tokenization_func(dataset): return tokenizer(dataset["input"])
tokenized_dataset = ds.map(tokenization_func, batched=True)

Map: 100%|██████████| 36473/36473 [00:00<00:00, 62574.84 examples/s]


This adds a new item to our dataset `input_ids` which converts our text in to numbers that match up with one of our models tokens. This list of tokens is known as the model's vocabulary.

In [30]:
row = tokenized_dataset[0]
row["input"], row["input_ids"]

('TEXT1: A47; TEXT2: abatement of pollution; ANC1: abatement',
 [1,
  54453,
  435,
  294,
  336,
  5753,
  346,
  54453,
  445,
  294,
  47284,
  265,
  6435,
  346,
  23702,
  435,
  294,
  47284,
  2])

In [31]:
tokenizer.convert_ids_to_tokens(54453)

'▁TEXT'

Transformers always assume that your labels are within a column named `labels` so we need to rename our score column.

In [32]:
tokenized_dataset = tokenized_dataset.rename_columns({'score':'labels'})

## Create a Validation Set

In practice a making a random split for a validation set is [often a bad practice](https://www.fast.ai/2017/11/13/validation-sets/). 

Note that for Kaggle competitions we use the training data for our validation set. The test data is for our test set.

In [33]:
ds_dict = tokenized_dataset.train_test_split(0.25, seed=42)
ds_dict

DatasetDict({
    train: Dataset({
        features: ['id', 'anchor', 'target', 'context', 'labels', 'input', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 27354
    })
    test: Dataset({
        features: ['id', 'anchor', 'target', 'context', 'labels', 'input', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 9119
    })
})

## Create a Test Set

In [34]:
test_df = pd.read_csv(path/"test.csv")
test_df["input"] = "TEXT1: " + test_df.context + "; TEXT2: " + test_df.target + "; ANC1: " + test_df.anchor
test_ds = Dataset.from_pandas(test_df).map(tokenization_func, batched=True)
test_ds

Map: 100%|██████████| 36/36 [00:00<00:00, 10342.83 examples/s]


Dataset({
    features: ['id', 'anchor', 'target', 'context', 'input', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 36
})

## Metrics
### Why not use the loss?
The loss function is used during the training process to quantify the difference between the predicted outputs and their actual targets. 

Metrics are used to evaluate the performance of the model.

The loss generally makes for a bad metric because it:
* Often doesn't align with our real world objectives
* Can be difficult to interprete to stakeholders
* Performs badly in imbalanced datasets where one class significantly outnumbers another
* Can't detect overfitting
* Cannot optimize for multiple objectives

### Correlation Coefficient
Choosing a metric [can be difficult ](https://www.fast.ai/posts/2019-09-24-metrics.html). Fortunately for Kaggle exercises a metric is provided for us. 

We'll be using the Correlation coefficient. It ranges from -1 to 1 for complete negative and positive correlation respectively.

In [ ]:
import numpy as np
def corr(x,y): return np.corrcoef(x,y)[0][1]
def corr_d(eval_pred): return {'pearson': corr(*eval_pred)}

## Training

The batch-size may need adjusting to fit the GPU we choose to use. If a memory crash occurs during training a lower batch size will be required.

Learning rate will also require some trial and error, like batch size the larger the value the faster we'll train but a value too large will result in failure.

In [35]:
from transformers import TrainingArguments, Trainer
learning_rate = 8e-5
epoch_count = 4
if is_kaggle:
    batch_size = 128
    fp16 = True
else:
    batch_size = 32
    fp16 = False

The arguments below should work fine in most cases, diving in to them isn't necessary at this stage

In [36]:
args = TrainingArguments('outputs', 
                         learning_rate=learning_rate,
                         warmup_ratio=0.1,
                         lr_scheduler_type='cosine',
                         fp16=fp16,
                         evaluation_strategy='epoch',
                         per_device_train_batch_size=batch_size,
                         per_device_eval_batch_size=batch_size*2,
                         num_train_epochs=epoch_count,
                         weight_decay=0.01,
                         report_to='none')

We can now create a `Trainer` which combines our model and data together. It will spit out a few warnings we can ignore.

In [37]:
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=1)

trainer = Trainer(model, 
                  args, 
                  train_dataset=ds_dict['train'], 
                  eval_dataset=ds_dict['test'],
                  tokenizer=tokenizer,
                  compute_metrics=corr_d)

Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-small and are newly initialized: ['pooler.dense.bias', 'classifier.weight', 'pooler.dense.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [38]:
trainer.train()

 15%|█▍        | 500/3420 [02:37<14:12,  3.42it/s]Checkpoint destination directory outputs/checkpoint-500 already exists and is non-empty.Saving will proceed but saved results may be invalid.


{'loss': 0.048, 'learning_rate': 7.948100247760808e-05, 'epoch': 0.58}


                                                    
 25%|██▌       | 855/3420 [04:57<23:20,  1.83it/s]

{'eval_loss': 0.026030583307147026, 'eval_pearson': 0.7970794374456811, 'eval_runtime': 24.2528, 'eval_samples_per_second': 375.998, 'eval_steps_per_second': 5.896, 'epoch': 1.0}


 29%|██▉       | 1000/3420 [05:43<11:23,  3.54it/s] Checkpoint destination directory outputs/checkpoint-1000 already exists and is non-empty.Saving will proceed but saved results may be invalid.


...

 88%|████████▊ | 3000/3420 [1:25:04<02:01,  3.47it/s]    Checkpoint destination directory outputs/checkpoint-3000 already exists and is non-empty.Saving will proceed but saved results may be invalid.


{'loss': 0.0101, 'learning_rate': 3.6193470242834415e-06, 'epoch': 3.51}


                                                         
100%|██████████| 3420/3420 [1:43:38<00:00,  1.82s/it]

{'eval_loss': 0.02154168114066124, 'eval_pearson': 0.8373890015435218, 'eval_runtime': 20.5034, 'eval_samples_per_second': 444.755, 'eval_steps_per_second': 6.974, 'epoch': 4.0}
{'train_runtime': 6218.375, 'train_samples_per_second': 17.596, 'train_steps_per_second': 0.55, 'train_loss': 0.021318977687791077, 'epoch': 4.0}


TrainOutput(global_step=3420, training_loss=0.021318977687791077, metrics={'train_runtime': 6218.375, 'train_samples_per_second': 17.596, 'train_steps_per_second': 0.55, 'train_loss': 0.021318977687791077, 'epoch': 4.0})

## Predictions
Now we've trained our model we can make some predictions

In [44]:
predictions = trainer.predict(test_ds).predictions.astype(float)
predictions

100%|██████████| 1/1 [00:00<00:00, 332.43it/s]


array([[ 0.47082999],
       [ 0.57536417],
       [ 0.50354177],
       [ 0.26576445],
       [-0.02382522],
       [ 0.55075079],
       [ 0.49101523],
       [-0.02531304],
       [ 0.29687223],
       [ 1.09018052],
       [ 0.22743776],
       [ 0.20256121],
       [ 0.78881073],
       [ 0.99500763],
       [ 0.7485072 ],
       [ 0.32892826],
       [ 0.28195688],
       [-0.0210176 ],
       [ 0.60453236],
       [ 0.36301944],
       [ 0.41035315],
       [ 0.26777557],
       [ 0.07301307],
       [ 0.18113427],
       [ 0.5371725 ],
       [-0.02387112],
       [-0.01914458],
       [-0.0300971 ],
       [-0.01788442],
       [ 0.55236644],
       [ 0.3317053 ],
       [ 0.00218474],
       [ 0.67597705],
       [ 0.52917236],
       [ 0.44183591],
       [ 0.22106177]])

Some of these predictions are outside the range of scores we can have (0-1) so we should trim them.

In [45]:
predictions = np.clip(predictions, 0, 1)
predictions

array([[0.47082999],
       [0.57536417],
       [0.50354177],
       [0.26576445],
       [0.        ],
       [0.55075079],
       [0.49101523],
       [0.        ],
       [0.29687223],
       [1.        ],
       [0.22743776],
       [0.20256121],
       [0.78881073],
       [0.99500763],
       [0.7485072 ],
       [0.32892826],
       [0.28195688],
       [0.        ],
       [0.60453236],
       [0.36301944],
       [0.41035315],
       [0.26777557],
       [0.07301307],
       [0.18113427],
       [0.5371725 ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.55236644],
       [0.3317053 ],
       [0.00218474],
       [0.67597705],
       [0.52917236],
       [0.44183591],
       [0.22106177]])

## Submit Results

In [48]:
submission = Dataset.from_dict({
    'id' : test_ds['id'],
    'score' : predictions
})
submission.to_csv('submission.csv', index=False)

Creating CSV from Arrow format: 100%|██████████| 1/1 [00:00<00:00, 276.85ba/s]


1022